In [18]:
%matplotlib inline

In [19]:
import pandas as pd
import numpy as np

In [20]:
%run clean_eval.py

In [21]:
import pylab

In [22]:
agrees = relations[relations['voteAvg'] >= 4]

In [23]:
joined_left = agrees.merge(relata, left_on='left', right_on='relatum', suffixes=('', '_left'))
joined = joined_left.merge(relata, left_on='right', right_on='relatum', suffixes=('', '_right')).fillna('')

In [24]:
nounsonly = joined #[joined['term-dominatPOS'].str.endswith('-n') & joined['term-dominatPOS_right'].str.endswith('-n')]

In [25]:
overlaps = nounsonly.groupby(['left', 'right'], sort=True).filter(lambda x: len(x) > 1)

In [26]:
examples = [nounsonly.ix[v[:10]] for k, v in nounsonly.groupby('relation').groups.iteritems()]

In [27]:
entailment_decisions = """
HasProperty	0	0
Synonym	1	1
MemberOf	0	0
HasA	1	0
MadeOf	1	0
Entails	1	0
IsA	1	0
Antonym	0	0
PartOf	0	1
""".strip().split("\n")
entailment_decisions = {x: (int(a), int(b)) for x, a, b in [y.split("\t") for y in entailment_decisions]}

In [28]:
entailments_data = {}
hypernymy_data = {}
for i, row in nounsonly.iterrows():
    left, right = row['left'], row['right']
    rel = row['relation']
    AentB, BentA = entailment_decisions[rel]
    entailments_data[(left, right)] = entailments_data.get((left, right), 0) | AentB
    entailments_data[(right, left)] = entailments_data.get((left, right), 0) | BentA
    hypernymy_data[(left, right)] = hypernymy_data.get((left, right), 0) | (rel == "IsA")
    hypernymy_data[(right, left)] = hypernymy_data.get((right, left), 0)

In [29]:
entailments_flattened = [{'left': l, 'right': r, 'entails': bool(e)} for (l, r), e in entailments_data.iteritems()]

In [30]:
hypernymy_flattened = [{'left': l, 'right': r, 'hyper': bool(e)} for (l, r), e in hypernymy_data.iteritems()]

In [31]:
pd.DataFrame(hypernymy_flattened).to_csv("/Users/roller/working/lexent/data/eval_hyper/data.tsv", sep="\t", index=False, header=False, columns=('left', 'right', 'hyper'))

In [32]:
pwd

u'/Users/roller/working/lexent/orig_data/eval'

In [33]:
pd.DataFrame(entailments_flattened).to_csv("/Users/roller/working/lexent/data/eval_entail/data.tsv", sep="\t", index=False, header=False, columns=('left', 'right', 'entails'))